In [60]:
import numpy as np
import plotly.graph_objs as go
import ipywidgets as widgets
from IPython.display import display, clear_output

In [61]:
# Grid and neighbor definition
shape = (100, 100, 100)
grid = np.zeros(shape, dtype=int)
seeds = []

# FCC array, change this as needed
neighbor_offsets = np.array([
    [ 1,  1,  0], [ 1, -1,  0], [-1,  1,  0], [-1, -1,  0],
    [ 1,  0,  1], [ 1,  0, -1], [-1,  0,  1], [-1,  0, -1],
    [ 0,  1,  1], [ 0,  1, -1], [ 0, -1,  1], [ 0, -1, -1]
])



In [62]:
# Add seed function
def add_seed(x, y, z):
    if 0 <= x < shape[0] and 0 <= y < shape[1] and 0 <= z < shape[2]:
        grid[x, y, z] = 1
        seeds.append((x, y, z))

# Strict grow function (safe per-site growth)
def grow(grid, n_crit=3, n_max=6, p=1.0):
    shape_x, shape_y, shape_z = grid.shape
    active_mask = (grid == 1)

    coord_count = np.zeros_like(grid, dtype=int)
    for dx, dy, dz in neighbor_offsets:
        shifted = np.roll(active_mask, shift=(dx, dy, dz), axis=(0, 1, 2))
        coord_count += shifted

    spreading_mask = (grid == 1) & (coord_count < n_max)
    saturated_mask = (grid == 1) & (coord_count >= n_max)

    neighbor_count = np.zeros_like(grid, dtype=int)
    saturated_neighbors = np.zeros_like(grid, dtype=int)
    for dx, dy, dz in neighbor_offsets:
        neighbor_count += np.roll(spreading_mask, shift=(dx, dy, dz), axis=(0, 1, 2))
        saturated_neighbors += np.roll(saturated_mask, shift=(dx, dy, dz), axis=(0, 1, 2))

    candidate_sites = (grid == 0) & (neighbor_count >= n_crit) & (saturated_neighbors == 0)
    xs, ys, zs = np.where(candidate_sites)

    new_grid = grid.copy()
    for idx in np.random.permutation(len(xs)):
        x, y, z = xs[idx], ys[idx], zs[idx]
        if np.random.rand() > p:
            continue

        new_self_coord = 0
        for dx, dy, dz in neighbor_offsets:
            nx, ny, nz = x + dx, y + dy, z + dz
            if 0 <= nx < shape_x and 0 <= ny < shape_y and 0 <= nz < shape_z:
                if new_grid[nx, ny, nz] == 1:
                    new_self_coord += 1
        if new_self_coord > n_max:
            continue

        violate = False
        for dx, dy, dz in neighbor_offsets:
            nx, ny, nz = x + dx, y + dy, z + dz
            if 0 <= nx < shape_x and 0 <= ny < shape_y and 0 <= nz < shape_z:
                if new_grid[nx, ny, nz] == 1:
                    if coord_count[nx, ny, nz] + 1 > n_max:
                        violate = True
                        break
        if violate:
            continue

        new_grid[x, y, z] = 1
        for dx, dy, dz in neighbor_offsets:
            nx, ny, nz = x + dx, y + dy, z + dz
            if 0 <= nx < shape_x and 0 <= ny < shape_y and 0 <= nz < shape_z:
                if new_grid[nx, ny, nz] == 1:
                    coord_count[nx, ny, nz] += 1
        coord_count[x, y, z] = new_self_coord

    return new_grid

# Plot both grid and suppressed sites

def plot_combined(grid, n_crit=3, n_max=6, show_active=True, show_suppressed=True):
    traces = []
    if show_active:
        active_mask = (grid == 1)
        coord_count = np.zeros_like(grid, dtype=int)
        for dx, dy, dz in neighbor_offsets:
            shifted = np.roll(active_mask, shift=(dx, dy, dz), axis=(0, 1, 2))
            coord_count += shifted

        pos_active = np.argwhere(grid == 1)
        colors = coord_count[grid == 1]
        hover_text = [f"Coordinate: ({x}, {y}, {z})<br>Coordination number: {c}" for (x, y, z), c in zip(pos_active, colors)]

        trace_active = go.Scatter3d(
            x=pos_active[:, 0], y=pos_active[:, 1], z=pos_active[:, 2],
            mode='markers',
            text=hover_text,
            hoverinfo='text',
            marker=dict(
                size=4,
                color=colors,
                colorscale='YlGnBu',
                cmin=0,
                cmax=12,
                colorbar=dict(title='Coord. Number')
            ),
            name='Active Sites'
        )
        traces.append(trace_active)

    if show_suppressed:
        suppressed = get_suppressed_sites(grid, n_crit, n_max)
        pos_sup = np.argwhere(suppressed)
        trace_suppressed = go.Scatter3d(
            x=pos_sup[:, 0], y=pos_sup[:, 1], z=pos_sup[:, 2],
            mode='markers',
            marker=dict(size=5, color='red', opacity=0.4),
            name='Suppressed'
        )
        traces.append(trace_suppressed)

    fig = go.Figure(data=traces)
    fig.update_layout(
        title='Active Sites and Suppressed Growth Locations',
        width=700, height=700,
        scene=dict(aspectmode='cube')
    )
    fig.show()

# Suppressed site mask
def get_suppressed_sites(grid, n_crit=2, n_max=6):
    shape_x, shape_y, shape_z = grid.shape
    active_mask = (grid == 1)

    coord_count = np.zeros_like(grid, dtype=int)
    for dx, dy, dz in neighbor_offsets:
        shifted = np.roll(active_mask, shift=(dx, dy, dz), axis=(0, 1, 2))
        coord_count += shifted

    spreading_mask = (grid == 1) & (coord_count < n_max)
    saturated_mask = (grid == 1) & (coord_count >= n_max)

    neighbor_count = np.zeros_like(grid, dtype=int)
    saturated_neighbors = np.zeros_like(grid, dtype=int)
    for dx, dy, dz in neighbor_offsets:
        neighbor_count += np.roll(spreading_mask, shift=(dx, dy, dz), axis=(0, 1, 2))
        saturated_neighbors += np.roll(saturated_mask, shift=(dx, dy, dz), axis=(0, 1, 2))

    suppressed_mask = (grid == 0) & (neighbor_count >= n_crit) & (saturated_neighbors > 0)
    return suppressed_mask

# UI controls
n_crit_slider = widgets.IntSlider(value=3, min=1, max=12, description='Min Neighbors')
n_max_slider = widgets.IntSlider(value=6, min=1, max=12, description='Max Neighbors')
p_slider = widgets.FloatSlider(value=1.0, min=0.0, max=1.0, step=0.05, description='Growth Prob')

step_button = widgets.Button(description='Grow Step')
reset_button = widgets.Button(description='Reset')
combined_button = widgets.Button(description='Show All')

show_active_checkbox = widgets.Checkbox(value=True, description="Show Active Sites")
show_suppressed_checkbox = widgets.Checkbox(value=True, description="Show Suppressed Sites")

run_output = widgets.Output()

# Event functions
def grow_step(b):
    global grid
    grid = grow(
        grid,
        n_crit=n_crit_slider.value,
        n_max=n_max_slider.value,
        p=p_slider.value
    )
    with run_output:
        clear_output(wait=True)
        plot_combined(
            grid,
            n_crit=n_crit_slider.value,
            n_max=n_max_slider.value,
            show_active=show_active_checkbox.value,
            show_suppressed=show_suppressed_checkbox.value
        )

def reset_grid(b):
    global grid, seeds
    grid[:] = 0
    seeds.clear()
    seed_custom()
    with run_output:
        clear_output(wait=True)
        plot_combined(
            grid,
            n_crit=n_crit_slider.value,
            n_max=n_max_slider.value,
            show_active=show_active_checkbox.value,
            show_suppressed=show_suppressed_checkbox.value
        )

def show_combined(b):
    with run_output:
        clear_output(wait=True)
        plot_combined(
            grid,
            n_crit=n_crit_slider.value,
            n_max=n_max_slider.value,
            show_active=show_active_checkbox.value,
            show_suppressed=show_suppressed_checkbox.value
        )

###################### Initialize seed #####################
def seed_custom():
    add_seed(shape[0]//2, shape[1]//2, shape[2]//2)
    add_seed(shape[0]//2+1, shape[1]//2+1, shape[2]//2)#3-member ring
    add_seed(shape[0]//2+1, shape[1]//2, shape[2]//2+1)#3-member ring
    #add_seed(shape[0]//2-1, shape[1]//2-1, shape[2]//2)#4-member ring
    #add_seed(shape[0]//2-2, shape[1]//2, shape[2]//2)#4-member ring
    #add_seed(shape[0]//2-1, shape[1]//2+1, shape[2]//2)#4-member ring

In [63]:
step_button.on_click(grow_step)
reset_button.on_click(reset_grid)
combined_button.on_click(show_combined)

seed_custom()
# UI layout
display(widgets.HBox([n_crit_slider, n_max_slider]))
display(widgets.HBox([p_slider]))
display(widgets.HBox([step_button, reset_button, combined_button]))
display(run_output)

plot_combined(grid)


Output()